In [16]:
import os
import sys
import shutil
import glob
import importlib
#from pathlib import Path
import random

import numpy as np
import librosa
import matplotlib.pyplot as plt
from sklearn.model_selection import train_test_split

import IPython

repos_dir = r'/home/akikun/repos'
sys.path.append(repos_dir)
from sak import display as dp
from sak import signal_processing as sp
from sak import nele

repos_imgan_dir = os.path.join(repos_dir, 'Intelligibility-MetricGAN')
sys.path.append(repos_imgan_dir)
import default_settings as default
#import audio_util as au

In [23]:
corpus_dir = r'/home/common/db/audio_corpora/nele'

corpus_clean_dir = os.path.join(corpus_dir, 'hikari_tts')
corpus_ssdrc_dir = os.path.join(corpus_dir, 'hikari_tts', 'wavs_ssdrc') 
corpus_noise_dir = os.path.join(corpus_dir, 'train_noise', 'JR')

data_dir = os.path.join(corpus_dir, 'imgan', 'per_snr')

experiments_dir = r'/home/akikun/experiments/jr'

n_test = 479
snr = 20

In [24]:
# make directories.
if not os.path.exists(data_dir):
    os.makedirs(data_dir)

train_dir = os.path.join(data_dir, 'train')
train_clean_dir = os.path.join(train_dir, 'clean')
train_noise_dir = os.path.join(train_dir, 'noise' + str(snr))
train_mixed_dir = os.path.join(train_dir, 'mixed' + str(snr))
train_enhanced_dir = os.path.join(train_dir, 'enhanced')

test_dir = os.path.join(data_dir, 'test')
test_clean_dir = os.path.join(test_dir, 'clean')
test_noise_dir = os.path.join(test_dir, 'noise' + str(snr))

if not os.path.exists(train_dir):
    os.makedirs(train_dir)
if not os.path.exists(train_clean_dir):
    os.makedirs(train_clean_dir)
if not os.path.exists(train_noise_dir):
    os.makedirs(train_noise_dir) 
if not os.path.exists(train_mixed_dir):
    os.makedirs(train_mixed_dir) 
if not os.path.exists(train_enhanced_dir):
    os.makedirs(train_enhanced_dir)
    
if not os.path.exists(test_dir):
    os.makedirs(test_dir)
if not os.path.exists(test_clean_dir):
    os.makedirs(test_clean_dir)
if not os.path.exists(test_noise_dir):
    os.makedirs(test_noise_dir) 

## train test split

In [19]:
# ## normalize the input data into RMS of 0.02.
# # clean
# #wav_paths = glob.glob(os.path.join(corpus_clean_dir, 'wavs', '*.wav'))
# # noise
# wav_paths = glob.glob(os.path.join(corpus_noise_dir, 'wavs', '*.wav'))

# for i, wav_path in enumerate(wav_paths, start=1):
#     print('\r' + '{}/{}'.format(i, len(wav_paths)), end="")
#     sp.normalize_rms_file(
#        wav_path, 
#        wav_path.replace('wavs', 'wavs_normalized'))

In [25]:
# train-test split.
clean_wavs = glob.glob(os.path.join(corpus_dir, 'hikari_tts', 'wavs_normalized', '*.wav'))
clean_train, clean_test = train_test_split(clean_wavs, test_size=n_test, random_state=1)

for wav_path in clean_train:
    shutil.copy(wav_path, os.path.join(train_clean_dir, os.path.basename(wav_path)))
    
for wav_path in clean_test:
    shutil.copy(wav_path, os.path.join(test_clean_dir, os.path.basename(wav_path)))

## Enhanced speech

In [26]:
# copy the wavs from wav_ssdrc to wav_enhanced dir.
# they should have the same ids to wavs in train/clean dir.
wav_clean_paths = glob.glob(os.path.join(train_clean_dir, '*.wav'))
for i, wav_clean_path in enumerate(wav_clean_paths, start=1):
    print('\r' + '{}/{}'.format(i, len(wav_clean_paths)), end="")
    wav_basename = os.path.basename(wav_clean_path)
    wav_enhanced_path = os.path.join(train_enhanced_dir, wav_basename)
    wav_ssdrc_path = os.path.join(corpus_ssdrc_dir, wav_basename)
    shutil.copy(wav_ssdrc_path, wav_enhanced_path)

6000/6000

In [7]:
dp.disp_wav_file(wav_enhanced_path)
dp.disp_wav_file(wav_enhanced_path.replace('enhanced', 'clean'))

## Create noise masks.

In [8]:
# ## list of noise data from JR.
# wav_paths = glob.glob(os.path.join(corpus_noise_dir, 'wavs_normalized', '*.wav'))
# wav_paths.sort()
# for wav_path in wav_paths:
#     print(os.path.basename(wav_path))
#     dp.disp_wav_file(wav_path)

In [27]:
wav_noise0_path = os.path.join(corpus_noise_dir, 'wavs_normalized', 'train_noise1.wav')

wav_clean_dirs = [train_clean_dir, test_clean_dir]
wav_noise_out_dirs = [train_noise_dir, test_noise_dir]
for wav_clean_dir, wav_noise_out_dir in zip(wav_clean_dirs, wav_noise_out_dirs):
    wav_clean_paths = glob.glob(os.path.join(wav_clean_dir, '*.wav'))
    wav_clean_paths.sort()
    
    for i, wav_clean_path in enumerate(wav_clean_paths, start=1):
        print('\r' + '{}/{}'.format(i, len(wav_clean_paths)), end="")
        wav_basename = os.path.basename(wav_clean_path)
        wav_mixed_path = os.path.join(
            train_mixed_dir, 
            wav_basename
        )
        nele.add_noise2(
            wav_clean_path, 
            wav_noise0_path, 
            wav_mixed_path, 
            wav_noise_out_path=os.path.join(wav_noise_out_dir, wav_basename), 
            snr=snr, 
            sampling_frequency=44100)

479/47900

In [ ]:
# wav_id = 'hikari-001_020.wav'
# wav_path_c = os.path.join(train_clean_dir, wav_id)
# wav_path_n = os.path.join(train_noise_dir, wav_id)
# wav_path_e = os.path.join(train_enhanced_dir, wav_id)
# wav_path_m = os.path.join(train_mixed_dir, wav_id)
# dp.disp_wav(wav_path_c)
# dp.disp_wav(wav_path_n)
# dp.disp_wav(wav_path_e)
# dp.disp_wav(wav_path_m)

## make mixed data set.
Above script makes the data of a specific SNR.  
Here those data will be mixed in given condition. 

In [31]:
## make combined dataset from dataset per SNR.

# per_snr has 6000 training data + 479 test data.
# how much will we use?
# resulting n_samples will be n_clean_{train, test} x len(snrs) 
n_clean_train = 1000
n_clean_test  = 100

snrs = [20, 15, 10, 5, 0, -5, -10]
dir_id = 'dataset_' + str(n_clean_train * len(snrs)) + '-' + str(n_clean_test * len(snrs))

data_dir_all = os.path.join(
    data_dir.replace('per_snr', 'all'), 
    dir_id)

In [32]:
train_dir_all = os.path.join(data_dir_all, 'train')
train_clean_dir_all = os.path.join(train_dir_all, 'clean')
train_noise_dir_all = os.path.join(train_dir_all, 'noise')
train_enhanced_dir_all = os.path.join(train_dir_all, 'enhanced')

test_dir_all = os.path.join(data_dir_all, 'test')
test_clean_dir_all = os.path.join(test_dir_all, 'clean')
test_noise_dir_all = os.path.join(test_dir_all, 'noise')

if not os.path.exists(train_dir_all):
    os.makedirs(train_dir_all)
if not os.path.exists(train_clean_dir_all):
    os.makedirs(train_clean_dir_all)
if not os.path.exists(train_noise_dir_all):
    os.makedirs(train_noise_dir_all) 
if not os.path.exists(train_enhanced_dir_all):
    os.makedirs(train_enhanced_dir_all)
    
if not os.path.exists(test_dir_all):
    os.makedirs(test_dir_all)
if not os.path.exists(test_clean_dir_all):
    os.makedirs(test_clean_dir_all)
if not os.path.exists(test_noise_dir_all):
    os.makedirs(test_noise_dir_all) 

In [33]:
# clean_dir = train_clean_dir
# n = n_clean_train

for train_kind in ['train', 'test']:
#for n, clean_dir in zip(n_clean, target_dir):
    if train_kind == 'train':
        clean_dir = train_clean_dir
        n = n_clean_train
    elif train_kind == 'test':
        clean_dir = test_clean_dir
        n = n_clean_test
        
    wav_paths = glob.glob(os.path.join(clean_dir, '*.wav'))
    wav_ids = [os.path.basename(path) for path in wav_paths[0:n]]

    for snr in snrs:
        print('{} - SNR {}'.format(train_kind, snr))

        for i, wav_id in enumerate(wav_ids, start=1):
            #print('\r' + '{}/{}'.format(i, len(wav_ids)), end="")
            wav_id_all = wav_id.replace('.wav', '_snr' + str(snr) + '.wav')

            # clean data.
            shutil.copy(
                os.path.join(clean_dir, wav_id),
                os.path.join(data_dir_all, train_kind, 'clean', wav_id_all)
            )

            # noise data.
            shutil.copy(
                os.path.join(clean_dir.replace('clean', 'noise') + str(snr), wav_id),
                os.path.join(data_dir_all, train_kind, 'noise', wav_id_all)
            )

            # enhanced data.
            if clean_dir == train_clean_dir:
                shutil.copy(
                    os.path.join(clean_dir.replace('clean', 'enhanced'), wav_id),
                    os.path.join(data_dir_all, train_kind, 'enhanced', wav_id_all)
                )

train - SNR 20
train - SNR 15
train - SNR 10
train - SNR 5
train - SNR 0
train - SNR -5
train - SNR -10
test - SNR 20
test - SNR 15
test - SNR 10
test - SNR 5
test - SNR 0
test - SNR -5
test - SNR -10


## make small dataset.

In [15]:
for root_dir in [train_dir_all, test_dir_all]:   
    print('>>> {}'.format(root_dir))
    root_dir_small = root_dir + '_small'

    for sub_dir_name in os.listdir(root_dir):
    #for sub_dir_name in ['noise-10']:
        sub_dir = os.path.join(root_dir_small, sub_dir_name)
        print(sub_dir_name)
        if os.path.exists(sub_dir):
            shutil.rmtree(sub_dir)
        os.makedirs(sub_dir)
        for wav_path in glob.glob(os.path.join(root_dir, sub_dir_name, 'hikari-001_*')):
            wav_basename = os.path.basename(wav_path)
            shutil.copy(
                wav_path,
                os.path.join(sub_dir, wav_basename)
            )

>>> /home/common/db/audio_corpora/nele/imgan/all/dataset_10000-500/train
clean
noise
enhanced
>>> /home/common/db/audio_corpora/nele/imgan/all/dataset_10000-500/test
clean
noise
